In [ ]:
import os
import pickle

In [8]:
directory = './code/results/citeseer/'
file_name = 'ind.citeseer.allx'
file_path = os.path.join(directory, file_name)
with open(file_path, 'rb') as f:
    data = pickle.load(f)
    print(f"File: {file_name}")
    if hasattr(data, 'shape'):
        print(f"Shape: {data.shape}")
    else:
        print("The loaded data does not have a 'shape' attribute.")

File: ind.citeseer.allx
Shape: torch.Size([3327, 3703])


In [10]:
directory = './code/datasets/citeseer/citeseer/raw/'
file_name = 'ind.citeseer.allx'
file_path = os.path.join(directory, file_name)
try:
    with open(file_path, 'rb') as f:
        data = pickle.load(f)  # Ensure binary mode is used
        print(f"File: {file_name}")
        if hasattr(data, 'shape'):
            print(f"Shape: {data.shape}")
        else:
            print("The loaded data does not have a 'shape' attribute.")
except UnicodeDecodeError as e:
    print(f"UnicodeDecodeError: {e}")
except FileNotFoundError:
    print(f"File not found: {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

UnicodeDecodeError: 'ascii' codec can't decode byte 0x85 in position 16: ordinal not in range(128)


In [17]:
import os
import pickle

# Specify the directory and file name
directory = './code/datasets/pubmed/pubmed/raw/'
file_name = 'ind.pubmed.allx'
file_path = os.path.join(directory, file_name)

# Open the file in binary mode and load it using pickle
try:
    with open(file_path, 'rb') as f:
        data = pickle.load(f)  # Ensure binary mode is used
        print(f"File: {file_name}")
        if hasattr(data, 'shape'):
            print(f"Shape: {data.shape}")
        else:
            print("The loaded data does not have a 'shape' attribute.")
except UnicodeDecodeError as e:
    print(f"UnicodeDecodeError: {e}")
except FileNotFoundError:
    print(f"File not found: {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

UnicodeDecodeError: 'ascii' codec can't decode byte 0xa1 in position 12: ordinal not in range(128)


In [ ]:
def load_dataset(
        dataset: dict(help='name of the dataset', option='-d', choices=supported_datasets) = 'cora',
        data_dir: dict(help='directory to store the dataset') = './datasets',
        data_range: dict(help='min and max feature value', nargs=2, type=float) = (0, 1),
        val_ratio: dict(help='fraction of nodes used for validation') = .25,
        test_ratio: dict(help='fraction of nodes used for test') = .25,
):
    data = supported_datasets[dataset](root=os.path.join(data_dir, dataset))
    data = RandomNodeSplit(split='train_rest', num_val=val_ratio, num_test=test_ratio)(data[0])
    data = ToSparseTensor()(data)
    data.name = dataset
    data.num_classes = int(data.y.max().item()) + 1

    if data_range is not None:
        low, high = data_range
        data = Normalize(low, high)(data)

    return data

# Load the BitcoinAlpha dataset
data = load_dataset(dataset='bitcoinalpha', data_dir='./code/datasets')

# Access the processed data
print(data)
print(f"Number of nodes: {data.num_nodes}")
print(f"Number of edges: {data.edge_index.size(1)}")
print(f"Feature matrix shape: {data.x.shape}")
print(f"Labels shape: {data.y.shape}")




















NameError: name 'load_dataset' is not defined

In [ ]:
#loading the bitcoinalpha dataset
 def process(self):
        import networkx as nx
        import numpy as np

        file_path = os.path.join(self.raw_dir, self.raw_file_names[0])
        df = pd.read_csv(file_path)

        # Step 1: Map nodes to a continuous ID range
        unique_nodes = pd.unique(df[['SOURCE', 'TARGET']].values.ravel())
        node_map = {node: idx for idx, node in enumerate(unique_nodes)}
        num_nodes = len(node_map)

        df['SOURCE'] = df['SOURCE'].map(node_map)
        df['TARGET'] = df['TARGET'].map(node_map)

        # Step 2: Build graph from remapped edges
        G = nx.from_pandas_edgelist(df, source='SOURCE', target='TARGET', edge_attr='RATING', create_using=nx.DiGraph())
        edge_index = torch.tensor(list(G.edges), dtype=torch.long).t()

        # Step 3: Generate features and labels
        max_rating = max(df['RATING'].abs())
        dmax_in = max(dict(G.in_degree()).values())
        dmax_out = max(dict(G.out_degree()).values())

        y = []
        x = []

        for node in range(num_nodes):
            in_edges = list(G.in_edges(node))
            out_edges = list(G.out_edges(node))

            # Label
            if len(in_edges) < 3:
                label = 0
            else:
                rating_sum = sum(G[u][v]['RATING'] / abs(G[u][v]['RATING']) for u, v in in_edges)
                avg_rating = rating_sum / len(in_edges)
                label = 1 if avg_rating >= 0.3 else 0
            y.append(label)

            # Features
            if len(out_edges) == 0:
                features = np.ones(8) / 1000
            else:
                w_pos = sum(G[u][v]['RATING'] for u, v in out_edges if G[u][v]['RATING'] > 0)
                w_neg = -sum(G[u][v]['RATING'] for u, v in out_edges if G[u][v]['RATING'] < 0)
                abstotal = w_pos + w_neg
                avg = (w_pos - w_neg) / len(out_edges) / max_rating

                f = np.zeros(8)
                f[0] = w_pos / max_rating / len(out_edges)
                f[1] = w_neg / max_rating / len(out_edges)
                f[2] = w_pos / abstotal if abstotal else 0
                f[3] = avg
                f[4] = f[0] * G.in_degree(node) / dmax_in if dmax_in else 0
                f[5] = f[1] * G.in_degree(node) / dmax_in if dmax_in else 0
                f[6] = f[0] * G.out_degree(node) / dmax_out if dmax_out else 0
                f[7] = f[1] * G.out_degree(node) / dmax_out if dmax_out else 0
                features = f / 1.01 + 0.001

            x.append(features)

        data = Data(
            x=torch.tensor(x, dtype=torch.float),
            edge_index=edge_index,
            y=torch.tensor(y, dtype=torch.long),
            num_nodes=num_nodes
        )

        if self.pre_transform is not None:
            data = self.pre_transform(data)

        torch.save(self.collate([data]), self.processed_paths[0])
